In [4]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from keras import optimizers
from keras import losses
from keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time, os, fnmatch, shutil

In [6]:
cmmd_dir = "F:/uni/UT/Y-Y/"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        cmmd_dir+"cmmd_data/PNG/TRAIN/",
        target_size=(227, 227),
        color_mode="grayscale",
        batch_size=8,
        shuffle=True,
        class_mode='binary',
        seed=14)
validation_generator = test_datagen.flow_from_directory(
        cmmd_dir+"cmmd_data/PNG/VAL/",
        target_size=(227, 227),
        color_mode="grayscale",
        batch_size=8,
        shuffle=True,
        class_mode='binary',
        seed=14)

Found 2402 images belonging to 2 classes.
Found 602 images belonging to 2 classes.


# Modified LeNet implementation

In [21]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(filters=6, 
                              kernel_size=(3, 3), 
                              activation='relu', 
                              input_shape=(256,256,1)))
model.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(filters=16, 
                              kernel_size=(3, 3), 
                              activation='relu'))
model.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=120, 
                             activation='relu'))

model.add(keras.layers.Dense(units=84, 
                             activation='relu'))

model.add(keras.layers.Dense(1, activation='sigmoid'))

# Uncomment for model summary
model.summary()

C:\Users\Amin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 6)    │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 127, 127, 6)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 62, 62, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 120)            │     7,380,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,391,789 (28.20 MB)

 Trainable params: 7,391,789 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    #TensorBosard run TITLE:
    run_id = ("LeNet_Candidate_Model")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [
    #comment next two lines to stop tensorboard if needed.
   # keras.callbacks.ModelCheckpoint("/media/craig/Henry/ML_models/lenet/save_at_{epoch}.h5"),
   # keras.callbacks.TensorBoard(run_logdir)
]

In [17]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
history = model.fit(train_generator, epochs=150, callbacks=callbacks, validation_data=validation_generator)

C:\Users\Amin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/150


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 61504, but received input with shape (None, 48400)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 227, 227, 1), dtype=float32)
  • training=True
  • mask=None

In [ ]:
model.save("lenet_candidate_model.h5")